# Applying pooling layers
A popular optimization technique for CNNs is pooling layers. A pooling layer is a method to reduce the number of trainable parameters in a smart way. Two of the most commonly used pooling layers are average pooling and maximum (max) pooling.<br> In the first, for a specified block size the inputs are averaged and extracted. For the latter, the maximum value within a block is extracted. These pooling layers provide a translational invariance. In other words, the exact location of a feature is less relevant. Also, by reducing the number of trainable parameters we limit the complexity of the network, which should prevent overfitting. Another benefit is that it will reduce the training and inference time significantly. <br> In the next recipe, we will add max pooling layers to the CNN we've implemented in the previous recipe and at the same time we will increase the number of filters in the convolutional layers.

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping

Using TensorFlow backend.


### Load the mnist dataset

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Reshape the training data to represent grayscaled image input

In [3]:
img_rows, img_cols = X_train[0].shape[0], X_train[0].shape[1]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

### Normalize the input data

In [4]:
X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

### One-hot encode the labels

In [5]:
n_classes = len(set(y_train))
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

### Define the CNN architecture and output the network architecture

In [6]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 256)         0         
__________

### Set the network hyperparameters  

In [7]:
batch_size = 128
n_epochs = 200

### define the callback function:

In [8]:
callbacks = [EarlyStopping(monitor='val_acc', patience=5)]

### Train the model

In [9]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_split=0.2, callbacks=callbacks)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 132s 3ms/step - loss: 0.3387 - acc: 0.8901 - val_loss: 0.0572 - val_acc: 0.9823
Epoch 2/200
48000/48000 [==============================] - 225s 5ms/step - loss: 0.1015 - acc: 0.9697 - val_loss: 0.0407 - val_acc: 0.9869
Epoch 3/200
48000/48000 [==============================] - 233s 5ms/step - loss: 0.0724 - acc: 0.9786 - val_loss: 0.0491 - val_acc: 0.9858
Epoch 4/200
48000/48000 [==============================] - 236s 5ms/step - loss: 0.0610 - acc: 0.9816 - val_loss: 0.0366 - val_acc: 0.9902
Epoch 5/200
48000/48000 [==============================] - 241s 5ms/step - loss: 0.0533 - acc: 0.9842 - val_loss: 0.0303 - val_acc: 0.9916
Epoch 6/200
48000/48000 [==============================] - 235s 5ms/step - loss: 0.0445 - acc: 0.9869 - val_loss: 0.0291 - val_acc: 0.9914
Epoch 7/200
48000/48000 [==============================] - 227s 5ms/step - loss: 0.0420 - acc: 0.9878 - val_loss: 0.

### Show the results on the test set

In [10]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0216482737543
Test accuracy: 0.9943


By applying a max pooling layer after each convolutional layer, we were able to decrease our error rate to 0.45 (a decrease of 36% compared to our previous error rate of 0.70). Moreover, by applying max pooling the number of trainable parameters is reduced significantly to 665,994. As a result, 1 epoch takes just around 11 seconds.